<a id='Table of Contents'></a><h1>Table of Contents</h1>
- <a href='#introduction'>Introduction</a>
- <a href='#imports'>Importing the required libraries</a>
- <a href='#anim_plot'>Plot of the fitting model</a>
- <a href='#ref_data'>Creating the reference data</a>
- <a href='#fitting'>Setting and fitting </a>  
- <a href='#plotting_results'>Plotting the results</a>  

(<a href='#Table of Contents'>Top</a>)<a id='introduction'></a><h2>Introduction</h2>



<div class="alert alert-block alert-info">
The objective of this notebook is to show how to use the <b>Equivalent Sites Circle</b> model to perform some 
fits using <a href="https://lmfit.github.io/lmfit-py/">lmfit</a>.
</div>

### Physical units
Please note that the following units are used for the QENS models

| Type of parameter | Unit          |
| ----------------- |---------------|
| Time              | picosecond    |
| Length            | Angstrom      |
| Momentum transfer | 1/Angstrom    |

(<a href='#Table of Contents'>Top</a>)<a id='imports'></a><h2>Importing the required librairies</h2>

In [ ]:
# install QENSmodels (if not already installed)
import pkgutil
import sys

if not pkgutil.find_loader("QENSmodels"):
    buttonY = pnw.Button(name='Yes', button_type='success')
    buttonN = pnw.Button(name='No', button_type='danger')
    choice_installation = panel.Column("Do you want to install the QENSmodels' library?", panel.Row(buttonY, buttonN))
    display(choice_installation)

In [ ]:
if not pkgutil.find_loader("QENSmodels"):
    if buttonY.clicks>0:
        !{sys.executable} -m pip install git+https://github.com/QENSlibrary/QENSmodels#egg=QENSmodels
    elif buttonN.clicks>0:
        print("You will not be able to run some of the remaining parts of this notebook")

In [ ]:
# import python modules for plotting, fitting
from __future__ import print_function
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

# for interactive plot
from pandas import DataFrame
import panel.widgets as pnw
import panel as pn
pn.extension()

# import model from QENS library
import QENSmodels

In [ ]:
# install lmfit (if not already installed)
if not pkgutil.find_loader("lmfit"):
    lmfitY = pnw.Button(name='Yes', button_type='success')
    lmfitN = pnw.Button(name='No', button_type='danger')
    choice_installation = panel.Column("Do you want to install lmfit?", panel.Row(lmfitY, lmfitN))
    display(choice_installation)

In [ ]:
if not pkgutil.find_loader("lmfit"):
    if lmfitY.clicks>0:
         !{sys.executable} -m pip install lmfit
    elif lmfitN.clicks>0:
        print("You will not be able to run some of the remaining parts of this notebook")

In [ ]:
# required imports from lmfit
from lmfit import Model, Parameters

(<a href='#Table of Contents'>Top</a>)<a id='anim_plot'></a><h2>Plot of the fitting model</h2>

The widget below shows the peak shape function imported from QENSmodels where the function's parameters can be varied.

In [ ]:
def mplplot(df, **kwargs):
    fig = df.plot(legend=False).get_figure()
    plt.grid()
    plt.ylabel('Equivalent Sites Circle')
    plt.xlabel('x')
    plt.close(fig)
    return fig

def equiv_sites_circle(q=1.0, scale=1.0, center=0.0, N=3, radius=1.0, resTime=1.0, view_fn=mplplot):
    xs = np.linspace(-10,10,100)
    ys = QENSmodels.sqwEquivalentSitesCircle(xs, q, scale, center, N, radius, resTime) 
    df = DataFrame(dict(y=ys), index=xs)
    return view_fn(df, q=q, scale=scale, center=center, N=N, radius=radius, resTime=resTime)

q = pnw.FloatSlider(name='q', value=1, start=0.2, end=10)
scale  = pnw.FloatSlider(name='scale', value=5, start=1., end=10)
center = pnw.FloatSlider(name='center', value=5, start=0, end=10)
N = pnw.IntSlider(name='N', value=3, start=2, end=20)
radius = pnw.FloatSlider(name='radius', value=5, start=0, end=10)
resTime =  pnw.FloatSlider(name='resTime', value=1, start=1, end=10)

def on_click(event):
    """Reset the interactive plots to inital values."""
    q.value = 1
    scale.value = 5
    center.value = 5
    N.value = 3
    radius.value = 5
    resTime.value = 1
    
reset_button = pnw.Button(name='Reset')

widgets    = pn.Column("#### Equivalent Sites Circle", q, scale, center, N, radius, resTime, reset_button)
site_equiv_panel = pn.Row(equiv_sites_circle(q.value, scale.value, center.value, N.value, radius.value, resTime.value), widgets)

def update(event):
    site_equiv_panel[0] = equiv_sites_circle(q.value, scale.value, center.value, N.value, radius.value, resTime.value)
    
q.param.watch(update, 'value')
scale.param.watch(update, 'value')
center.param.watch(update, 'value')
N.param.watch(update, 'value')
radius.param.watch(update, 'value')
resTime.param.watch(update, 'value')

reset_button.param.watch(on_click, 'clicks')

site_equiv_panel

(<a href='#Table of Contents'>Top</a>)<a id='ref_data'></a><h2>Creating the reference data</h2>

In [ ]:
xx = np.linspace(-5,5,100)
equiv_sites_circle_noisy = QENSmodels.sqwEquivalentSitesCircle(xx, 1., 1.3, 0.3, 5, 0.25, 0.4)*(1+0.1*np.random.normal(0,1,100)) + 0.01*np.random.normal(0,1,100)

(<a href='#Table of Contents'>Top</a>)<a id='fitting'></a><h2>Setting and fitting</h2>

In [ ]:
gmodel = Model(QENSmodels.sqwEquivalentSitesCircle)

print('Names of parameters:', gmodel.param_names)
print('Independent variable(s):', gmodel.independent_vars)

initial_parameters_values = [1.22, 0.25, 5, 0.3, 0.33]

# Define boundaries for parameters to be refined
gmodel.set_param_hint('scale', min=0)
gmodel.set_param_hint('center', min=-5, max=5)
gmodel.set_param_hint('radius', min=0)
gmodel.set_param_hint('resTime', min=0)

# Fix some of the parameters
gmodel.set_param_hint('q', vary=False)
gmodel.set_param_hint('N', vary=False)


# Fit
result = gmodel.fit(equiv_sites_circle_noisy, w=xx, q=1.,
                    scale=initial_parameters_values[0], 
                    center=initial_parameters_values[1],
                    N=initial_parameters_values[2],
                    radius=initial_parameters_values[3],
                    resTime=initial_parameters_values[4])

# Plots
# to increase vertical spacing between plots
plt.subplots_adjust(left=None, bottom=None, right=None, top=None,
                wspace=None, hspace=0.75)

# Initial model and reference data
plt.subplot(2, 1, 1)
plt.plot(xx, equiv_sites_circle_noisy, 'b-', label='reference data')
plt.plot(xx, result.init_fit, 'k--', label='model with initial guesses')
plt.xlabel('x')
plt.grid()
plt.title('Initial model and reference data')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

# Fitting result and reference data
plt.subplot(2, 1, 2)
plt.plot(xx, equiv_sites_circle_noisy, 'b-', label='reference data')
plt.plot(xx, result.best_fit, 'r.', label='fitting result')
plt.title('Fit result and reference data')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlabel('x')
plt.grid()

plt.show()

(<a href='#Table of Contents'>Top</a>)<a id='plotting_results'></a><h2>Plotting results </h2>
using methods implemented in `lmfit`

In [ ]:
# display result
print('Result of fit:\n',result.fit_report())

# plot
result.plot()